# A 1D-cutting stock example

### Description of the problem
A small company has been ordered to cut table legs for school desks. The school desks are of 3 different sizes (Small, Medium and Large) and differ only by the length of the table legs (S=40cm, M=60cm, L=70cm). The number of desks (with four legs each) required is 108 for size S, 125 for size M and 100 for size L. To make the table legs, you will cut master metal tubes of unique size (2m) and you always have enough tube to satisfy the demand.

<img src="table-tubes.png" style="width:800px" />

**Problem**: How are you going to cut the master metal tubes in order to answer the demand and minimize the waste (the loss of all cut master metal tube).

### Data files for resolution

In [ ]:
;cat Data/inst1-tablelegs.dat

In [ ]:
;cat Data/inst1-tubesize.dat

## ILP model for cutting stock (with patterns)

### Data for the model
We have the following data:
- $T\ \ \ \ \;$ tube size
- $L\ \ \ \ \;$ table leg size (vector 1..$nf$)
- $D\ \ \ \ $ table leg demand (vector 1..$nf$)
- $n\ \ \ \ \ $ number of different patterns
- $m\ \ \ \ $ pattern composition ($m_{ij}$ number of table legs of size $j$ is cut in pattern $i$) (array 1..$n$,1..$nf$)
- $c\ \ \ \ $ waste for each pattern (vector 1..$n$)

### Model

\begin{align*}
  \min \quad & \sum_{i=1}^n c_ix_i                      \\
  s.t. \quad & \nonumber                                                 \\
             & \sum_{i=1}^n m_{ij}x_i = d_j \quad & \forall j \in \{1..m\} \\
             & x_i \in \mathbb{Z}^+ \quad & \forall i \in \{1..n\}
\end{align*}

## Let's play with the code

We need the following steps:
- Read data
- Generate patterns
- Build the math model
- Solve it
- Display results

### Read data

In [ ]:
using DelimitedFiles; # reading delimited files (can be replaced by CSV)
using Printf;         # for expert only ;-)

dir="Data/"
prefix="inst1"

LD  = readdlm(dir*prefix*"-tablelegs.dat",',',Int,comments=true); # table legs 
T  = readdlm(dir*prefix*"-tubesize.dat",',',Int,comments=true)[1]; # tube size

L = LD[1,:];    # isolate table leg length
D = LD[2,:];    # isolate table leg demand
nf = length(L); # number of different formats

### Generate patterns

In [ ]:
MAXN = 1 # maximum number of patterns
for j = 1:nf
    global MAXN
    MAXN = MAXN * (1+div(T,L[j]))
end
MAXN < 0 ? MAXN = 1000000 : MAXN # ensure non-negative value
print(MAXN);

In [ ]:
# declare new arrays
m = zeros(Int,MAXN,nf); # composition of patterns
c = zeros(Int,MAXN);    # waste for each pattern
gap = zeros(Int,nf);    # sort of hashing function
Tmp = zeros(Int,nf);    # temporary patterns

In [ ]:
# compute all patterns and waste (similar code to a hashing code)
println("-- Patterns -----------------")
gap[nf] = 1 # computing gap(k)
for k = 1:nf-1
    gap[nf-k] = gap[nf-k+1]*(1+div(T,L[nf-k+1]))
end
p = 0 # index of valid patterns
for i = 1:MAXN
    global p
    temp = i
    for k = 1:nf
        Tmp[k] = div(temp,gap[k])
        temp = temp - gap[k]*Tmp[k]
    end
    # computing pattern length
    PL = sum(L[j]*Tmp[j] for j =1:nf)
    # the current pattern is stored in m if PL <= T
    if PL <= T 
        p = p+1
        c[p] = T - PL
        m[p,:] = Tmp 
        #println("-- id ",p," Waste = ", c[p]," :: ",m[p,:])
        @printf("%s%3d%s%4d%s","-- id ",p," Waste = ", c[p]," :: ")
        println(m[p,:])
    end
end
println("-----------------------------")

In [ ]:
### save patterns to file
open(dir*prefix*"-patterns.dat", "w") do io
    println(io,"# All patterns generated for ",prefix," data")
    writedlm(io, m[1:p,:], ',')
end

### Build the math model

In [ ]:
using DelimitedFiles; # read files in csv format
using Printf;         # who can skip that package?
using JuMP;           # math programmaing interface
using HiGHS;          # HiGHS solver
# using Gurobi;       # one of the best commercial solver
# using Cbc;          # COIN-OR solver 

In [ ]:
# Re-reading data (to have them clean)
dir="Data/"
prefix="inst1"; # prefix for all files used here

### Reading data
println("-- Data -------------------------")
LD  = readdlm(dir*prefix*"-tablelegs.dat",',',Int,comments=true); # table legs 
T  = readdlm(dir*prefix*"-tubesize.dat",',',Int,comments=true)[1]; # tube size

L = LD[1,:];    # isolate table leg length
D = LD[2,:];    # isolate table legs demand
nf = length(L); # number of different formats
println("-- Table leg size")
print("   "); println(L)
println("-- Table leg demand")
print("   "); println(D)
println("-- Tube size")
print("   "); println(T)

m = readdlm(dir*prefix*"-patterns.dat",',',Int,comments=true);     # current list of patterns
n = size(m,1);         # number of pattern read
c = T*ones(Int,n)-m*L; # recalculate the waste for each pattern

println();
println("-- Patterns ---------------------")
for p=1:n
    @printf("%s%3d%s%4d%s","-- id ",p," Waste = ", c[p]," :: ")
    println(m[p,:])
end

In [ ]:
### Modelling the problem

# select my current solver
solverSelected = HiGHS.Optimizer;

# model
cs = Model(solverSelected); # declare the cg model

# variables -- number of time pattern i is used
@variable(cs, x[1:n] >= 0, Int);

# constraints -- satisfy the demand exactly
@constraint(cs, Demand[j=1:nf], sum(m[i,j]*x[i] for i=1:n) == D[j]);

# objective -- minimize total waste
@objective(cs, Min, sum(c[i]*x[i] for i=1:n));

### Remember the model

\begin{align*}
  \min \quad & \sum_{i=1}^n c_ix_i                      \\
  s.t. \quad & \nonumber                                                 \\
             & \sum_{i=1}^n m_{ij}x_i = d_j \quad & \forall j \in \{1..m\} \\
             & x_i \in \mathbb{Z}^+ \quad & \forall i \in \{1..n\}
\end{align*}

### Solving the model

In [ ]:
### Solving the model
println();
println("-- ILP resolution ---------------")
optimize!(cs) # solve command

In [ ]:
println("TERMIMATION STATUS: ",termination_status(cs)) # check the status of the resolution, ALWAYS!!!

### Printing results

In [ ]:
### Printing results
println();
println("-- Results ----------------------")
# Total waste
println("  Number of tubes used: ", Int(sum(value.(x))))
println("  Total waste: ",objective_value(cs)," cm")
for i=1:n
    if value(x[i])!=0
        @printf("  Pattern %3d: %s used %3d times, waste %3d cm\n",i,m[i,:],Int(value(x[i])),c[i]*value(x[i]));
    end
end